In [37]:
import csv
import requests as rq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from pandas import Series
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, Activation
import tensorflow as tf
from keras.layers import Masking
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.efficient_frontier import EfficientCVaR
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
from pypfopt.efficient_frontier import EfficientSemivariance
import copy
from pypfopt.risk_models import CovarianceShrinkage
from matplotlib.ticker import FuncFormatter
import dash
from dash import Dash, dcc, html, Input, Output, dash_table
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash.dependencies as dd
from datetime import datetime
import plotly.graph_objs as go
import plotly.express as px

In [38]:
url = 'https://www.slickcharts.com/sp500'
params = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
res = rq.get(url,headers = params)

In [39]:
data = res.text

In [40]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(data,'lxml')

In [41]:
companies = []
for i in range(0,50,1):
    companies.append(soup.body.find_all('div')[8].tbody.find_all('tr')[i].find_all('a')[1].text)

In [42]:
companies[companies.index('BRK.B')] = 'BRK-B'

In [43]:
url = 'https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?period1=1619222400&period2=1650844800&interval=1d&events=history&includeAdjustedClose=true'
sp500 = pd.read_csv(url)

In [44]:
datetime.now().timestamp()-(int(datetime(2018, 4, 25).timestamp())-int(datetime(2018, 4, 24).timestamp()))

1652343778.038464

In [45]:
period1 = int(datetime.now().timestamp()-(int(datetime(2022, 4, 25).timestamp())-int(datetime(2018, 4, 24).timestamp())))
period2 = int(datetime.now().timestamp()-(int(datetime(2018, 4, 25).timestamp())-int(datetime(2018, 4, 24).timestamp())))
stocks = pd.DataFrame()
prices = pd.DataFrame()
a = 0
b = 0
for i in companies:
    CSV_URL = f'https://query1.finance.yahoo.com/v7/finance/download/{i}?period1={period1}&period2={period2}&interval=1d&events=history&includeAdjustedClose=true'
    data = pd.read_csv(CSV_URL)
    prices[i] = data['Close']
    stocks[i] = data['Close']
    a = a+1
    if a<7:
        continue
    else:
        a = 0
        b = b + 1
    print(a,b)
for i in prices:
    prices[i] = np.insert(np.diff(np.log(prices[i])),0,0)

0 1
0 2
0 3
0 4
0 5
0 6
0 7


In [46]:
X_train = prices[:1008//2].transpose()
y_train = prices[1008//2:756].mean()
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from pandas import Series
for i in X_train:
    series = Series(X_train[i])
    values = series.values
    values = values.reshape((len(values), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(values)
    normalized = scaler.transform(values)
    X_train[i] = normalized
X_train = X_train.values.reshape(-1,1, 504)
#X_test  = X_test.values.reshape(-1, 1,14)
y_train = np.array(y_train).reshape(-1,1,1)
#y_test = np.array(y_test).reshape(-1, 1, 3)

model = Sequential()
#model.add(Masking(-1, batch_input_shape=(None,1,14)))

model.add(LSTM(400,return_sequences=True,activation='tanh',batch_input_shape=(None,1, 504)))
model.add(Dropout(0.02))

model.add(LSTM(200,activation='tanh',return_sequences=True))
#model.add(Dropout(0.05))

model.add(LSTM(100,activation='tanh',return_sequences=True))
#model.add(Dropout(0.05))

model.add(LSTM(50,activation='tanh',return_sequences=True))
#model.add(Dropout(0.05))

model.add(LSTM(14, activation='tanh',return_sequences=True))
#model.add(Dropout(0.01))
#model.add(LSTM(2, activation='tanh'))
model.add(Dense(6,activation='tanh'))

model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss=tf.keras.losses.MeanAbsolutePercentageError(
    name='mean_absolute_percentage_error'
),
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3,decay=1e-7)
              , metrics=['mse'])
model.fit(X_train,y_train, epochs=400, batch_size=20)
X_test = prices[(len(prices)-756):(len(prices)-252)].transpose()
for i in X_test:
    series = Series(X_test[i])
    values = series.values
    values = values.reshape((len(values), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(values)
    normalized = scaler.transform(values)
    X_test[i] = normalized
X_test = X_test.values.reshape(-1,1, 504)
y_test = prices[756:].mean()
y_test = np.array(y_test).reshape(-1,1,1)
model.evaluate(X_test,y_test)

Epoch 1/400
3/3 [==============================] - 15s 26ms/step - loss: 420.0159 - mse: 1.7396e-05
Epoch 2/400
3/3 [==============================] - 0s 26ms/step - loss: 621.3472 - mse: 8.4557e-06
Epoch 3/400
3/3 [==============================] - 0s 24ms/step - loss: 182.9192 - mse: 3.9455e-06
Epoch 4/400
3/3 [==============================] - 0s 24ms/step - loss: 232.7929 - mse: 1.0825e-06
Epoch 5/400
3/3 [==============================] - 0s 25ms/step - loss: 149.7521 - mse: 2.3493e-06
Epoch 6/400
3/3 [==============================] - 0s 23ms/step - loss: 234.0830 - mse: 5.3836e-06
Epoch 7/400
3/3 [==============================] - 0s 25ms/step - loss: 101.9371 - mse: 1.2972e-06
Epoch 8/400
3/3 [==============================] - 0s 23ms/step - loss: 138.1773 - mse: 9.3453e-07
Epoch 9/400
3/3 [==============================] - 0s 25ms/step - loss: 99.3188 - mse: 2.5035e-06
Epoch 10/400
3/3 [==============================] - 0s 22ms/step - loss: 115.4572 - mse: 1.0751e-06
Epoch 11/

3/3 [==============================] - 0s 27ms/step - loss: 128.6702 - mse: 6.1125e-07
Epoch 84/400
3/3 [==============================] - 0s 22ms/step - loss: 69.7367 - mse: 1.0316e-06
Epoch 85/400
3/3 [==============================] - 0s 25ms/step - loss: 114.1542 - mse: 5.9286e-07
Epoch 86/400
3/3 [==============================] - 0s 24ms/step - loss: 109.0364 - mse: 1.1228e-06
Epoch 87/400
3/3 [==============================] - 0s 22ms/step - loss: 160.7214 - mse: 2.7089e-06
Epoch 88/400
3/3 [==============================] - 0s 21ms/step - loss: 80.5733 - mse: 7.9042e-07
Epoch 89/400
3/3 [==============================] - 0s 25ms/step - loss: 275.4482 - mse: 1.9352e-06
Epoch 90/400
3/3 [==============================] - 0s 24ms/step - loss: 139.1389 - mse: 8.4958e-07
Epoch 91/400
3/3 [==============================] - 0s 23ms/step - loss: 125.4699 - mse: 2.7978e-06
Epoch 92/400
3/3 [==============================] - 0s 26ms/step - loss: 108.5759 - mse: 2.2946e-06
Epoch 93/400
3/

3/3 [==============================] - 0s 24ms/step - loss: 63.1203 - mse: 2.4381e-07
Epoch 166/400
3/3 [==============================] - 0s 24ms/step - loss: 37.6977 - mse: 4.1073e-07
Epoch 167/400
3/3 [==============================] - 0s 20ms/step - loss: 31.5127 - mse: 2.5893e-07
Epoch 168/400
3/3 [==============================] - 0s 26ms/step - loss: 30.6135 - mse: 1.3598e-07
Epoch 169/400
3/3 [==============================] - 0s 25ms/step - loss: 39.4962 - mse: 1.9915e-07
Epoch 170/400
3/3 [==============================] - 0s 22ms/step - loss: 30.6800 - mse: 2.3195e-07
Epoch 171/400
3/3 [==============================] - 0s 21ms/step - loss: 36.6213 - mse: 2.1525e-07
Epoch 172/400
3/3 [==============================] - 0s 26ms/step - loss: 52.0422 - mse: 2.7293e-07
Epoch 173/400
3/3 [==============================] - 0s 21ms/step - loss: 93.2616 - mse: 8.4629e-07
Epoch 174/400
3/3 [==============================] - 0s 22ms/step - loss: 54.3076 - mse: 2.2500e-07
Epoch 175/400


Epoch 247/400
3/3 [==============================] - 0s 24ms/step - loss: 68.8944 - mse: 2.7124e-07
Epoch 248/400
3/3 [==============================] - 0s 26ms/step - loss: 61.7338 - mse: 1.8884e-07
Epoch 249/400
3/3 [==============================] - 0s 21ms/step - loss: 136.0281 - mse: 9.4062e-07
Epoch 250/400
3/3 [==============================] - 0s 24ms/step - loss: 26.6215 - mse: 1.6358e-07
Epoch 251/400
3/3 [==============================] - 0s 26ms/step - loss: 30.2397 - mse: 8.4041e-08
Epoch 252/400
3/3 [==============================] - 0s 24ms/step - loss: 59.8821 - mse: 1.4032e-07
Epoch 253/400
3/3 [==============================] - 0s 23ms/step - loss: 79.7976 - mse: 5.1433e-07
Epoch 254/400
3/3 [==============================] - 0s 25ms/step - loss: 52.9770 - mse: 2.8963e-07
Epoch 255/400
3/3 [==============================] - 0s 25ms/step - loss: 60.1045 - mse: 3.4922e-07
Epoch 256/400
3/3 [==============================] - 0s 24ms/step - loss: 75.9964 - mse: 1.4452e-07

3/3 [==============================] - 0s 23ms/step - loss: 37.0306 - mse: 5.2577e-08
Epoch 329/400
3/3 [==============================] - 0s 25ms/step - loss: 35.7999 - mse: 3.6914e-07
Epoch 330/400
3/3 [==============================] - 0s 23ms/step - loss: 45.5009 - mse: 1.1735e-07
Epoch 331/400
3/3 [==============================] - 0s 20ms/step - loss: 25.2767 - mse: 1.4847e-07
Epoch 332/400
3/3 [==============================] - 0s 25ms/step - loss: 69.4792 - mse: 2.1314e-07
Epoch 333/400
3/3 [==============================] - 0s 21ms/step - loss: 81.7473 - mse: 8.4955e-07
Epoch 334/400
3/3 [==============================] - 0s 24ms/step - loss: 45.8883 - mse: 2.3872e-07
Epoch 335/400
3/3 [==============================] - 0s 24ms/step - loss: 149.1869 - mse: 8.5327e-07
Epoch 336/400
3/3 [==============================] - 0s 25ms/step - loss: 44.0267 - mse: 1.5600e-07
Epoch 337/400
3/3 [==============================] - 0s 22ms/step - loss: 170.2922 - mse: 9.8454e-07
Epoch 338/40

[902.4650268554688, 4.379337951831985e-06]

In [47]:
#plt.plot([i[0][0] for i in (model.predict(X_test)-y_test)]-np.mean([i[0][0] for i in (model.predict(X_test)-y_test)]))
robust = np.mean([i[0][0] for i in (model.predict(X_test)-y_test)])
X_test = prices[(len(prices)-504):(len(prices))].transpose()
for i in X_test:
    series = Series(X_test[i])
    values = series.values
    values = values.reshape((len(values), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(values)
    normalized = scaler.transform(values)
    X_test[i] = normalized
X_test = X_test.values.reshape(-1,1, 504)
mu = pd.DataFrame()
mu['return'] = [np.round(x[0][0]*365,3) for x in (model.predict(X_test)-robust)]
mu.index = prices.columns
mu = mu.squeeze()

In [53]:
def shares(a,b,c,d):
    rows = []
    time0 = datetime.now()
    if a==3:
        data_prek1 = pd.DataFrame()
        for i in stocks.iloc[:,:25]:
            for j in stocks.iloc[:,:25].drop(rows,axis=1):
                for w in stocks.iloc[:,:25].drop(rows,axis=1):
                    if i!=j and i!=w and j!=w:
                        try:
                            df_stocks = stocks[[i,j,w]]
                            m = mu[[i,j,w]]
                            S = expected_returns.returns_from_prices(df_stocks)
                            es = EfficientCVaR(m, S,weight_bounds=(0,1),beta=b)
                            es.efficient_return(c,market_neutral=False)
                            sharpe_pwt=es
                            data_prek1[str(i+'_'+j+'_'+w)] = [np.round(sharpe_pwt.portfolio_performance()[0],4),np.round(sharpe_pwt.portfolio_performance()[1],4),np.round(sharpe_pwt.clean_weights()[i],3),np.round(sharpe_pwt.clean_weights()[j],3),np.round(sharpe_pwt.clean_weights()[w],3)]
                        except:
                            continue
                rows.append(i)
                print(i)
        score = data_prek1.transpose()
        score.columns = ['return','volatility','weight1','weight2','weight3']
    elif a==2:
        data_prek1 = pd.DataFrame()
        for i in stocks.iloc[:,:25]:
            for j in stocks.iloc[:,:25].drop(rows,axis=1):
                if i!=j:
                    try:
                        df_stocks = stocks[[i,j]]
                        m = mu[[i,j]]
                        S = expected_returns.returns_from_prices(df_stocks)
                        es = EfficientCVaR(m, S,weight_bounds=(0,1),beta=b)
                        es.efficient_return(c,market_neutral=False)
                        #es.min_volatility(market_neutral=True)
                        sharpe_pwt=es
                        data_prek1[str(i+'_'+j)] = [np.round(sharpe_pwt.portfolio_performance()[0],4),np.round(sharpe_pwt.portfolio_performance()[1],4),np.round(sharpe_pwt.clean_weights()[i],3),np.round(sharpe_pwt.clean_weights()[j],3)]
                    except:
                        continue
            rows.append(i)
        score = data_prek1.transpose()
        score.columns = ['return','volatility','weight1','weight2']
    else:
        data_prek1 = pd.DataFrame()
        for i in stocks.iloc[:,:50]:
            try:
                df_stocks = stocks[[i]]
                m = mu[[i]]
                S = expected_returns.returns_from_prices(df_stocks)
                es = EfficientCVaR(m, S,weight_bounds=(0,1),beta=b)
                es.efficient_return(c,market_neutral=False)
                sharpe_pwt=es
                data_prek1[str(i)] = [np.round(sharpe_pwt.portfolio_performance()[0],4),np.round(sharpe_pwt.portfolio_performance()[1],4),sharpe_pwt.clean_weights()[i]]
            except:
                continue
        score = data_prek1.transpose()
        score.columns = ['return','volatility','weight1']
    if d=='Низкий':
        score1 = pd.DataFrame(score[score['volatility']==min(score['volatility'])].iloc[0]).transpose()
    elif d=='Средний':
        score1 = score[(np.quantile(score['return'],0.25)<=list(score['return']))&(np.quantile(score['return'],0.75)>=list(score['return']))]
        score1 = pd.DataFrame(score1[score1['volatility']==min(score1['volatility'])].iloc[0]).transpose()
    else:
        score1 = pd.DataFrame(score['return'].sort_values('return',ascending=False).iloc[0]).transpose()
    col = ['return','volatility']
    for i in score1.index[0].split('_'):
        col.append(i)
    score1.columns = col
    time1 = datetime.now()
    print(time1-time0)
    return score1

In [54]:
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
        "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]

In [60]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

risk = ['Консервативная', 'Умеренная', 'Повышенная']
risk1 = [0.99,0.95,0.8]
quantity = [1,2,3]
evaluation = ['Низкий','Средний','Высокий']


app.layout = html.Div([
    html.H1(children="Инвестиционный портфель для начинающих", className="header-title",
            style = {"font-family": "Helvetica, Verdana, sans-serif", "font-size": "50px","color": "darkslategrey","margin" :"2px auto", "text-align": "center", "text-shadow": "1px 1px 1px grey"}),
    html.P(children="Поможем собрать портфель акций с оптимальным соотношением надежности и доходности", className="header-description",
            style = { "font-family": "Helvetica","text-align":"center", 'padding-top' : "1%"}),
    
    html.Img(src ="https://i.ibb.co/Cn5F8Ym/image.jpg",
             style={"width": "60%","margin-top": "110px","vertical-align": "middle"}),  

    html.P("Оценка риска:", style = { "font-family": "Helvetica", "color": "darkslategrey","font-weight": "bold"}),
   dcc.Dropdown(
       id="risk",
       options=[{'label':r,'value':k}
               for r,k in zip(risk,risk1)],
       value=0.95,
       style = {'width': "71%"},
       clearable=False,
   ),
    html.P("Уровень риска:",style = { "font-family": "Helvetica","color": "darkslategrey","font-weight": "bold"}),
    dcc.Dropdown(
       id="evaluation",
       options=[{'label':risk,'value':risk}
               for risk in evaluation],
       value='Низкий',
       style = {'width': "71%","font-family": "Helvetica"},
       clearable=False,
   ),
    html.P("Выбор доходности:",style = { "font-family": "Helvetica","color": "darkslategrey","font-weight": "bold"}), 
    dcc.Input(
        id="return",
        type = 'number',
        min=0, max=max(mu),
        value=0.05,
        style = {'width': "50%","height": "4vh", "margin" :"2px auto","font-family": "Helvetica","font-size": "15px"},
        step=0.01,
    ),
    html.P("Количество акций в портфеле:",style = { "font-family": "Helvetica","color": "darkslategrey","font-weight": "bold"}), 
   dcc.Dropdown(
       id="quant",
       options=[{'label':quant,'value':quant}
               for quant in  quantity],
       value=2,
       style = {'width': "71%","font-family": "Helvetica"},
       clearable=False,
   ),
    
    dbc.Button(id='button',type='submit', children="Найти", style = {'backgroundColor': 'ghostwhite','color':'darkslategrey','border':'1.5px darkslategrey solid', "font-family": "Helvetica",'width': "8%", "height": "4vh", "margin" :"6px auto","font-weight": "bold"}),
    dash_table.DataTable(style_cell={
        'overflow': 'hidden',
        'textOverflow': 'ellipsis',
        'maxWidth': 0
    },
    id='output')
])



@app.callback(
    [dd.Output("output","data"),dd.Output('output','columns')], 
    [dd.State("quant", "value"),
    dd.State("risk", "value"),
    dd.State("return", "value"),
    dd.State("evaluation", "value"),
    dd.Input('button','n_clicks')])



def Table(quant,risk,returnn,v,n):
    if n is not None:
        new_data = shares(quant,risk,returnn,v)
        if len(new_data)>0:
            columns = [{"name": i, "id": i} for i in new_data.columns]
            data = new_data.to_dict('records')
            return data, columns
        else:
            raise dash.exceptions.PreventUpdate
        n = 0


app.run_server(use_reloader=False,debug=True,host='0.0.0.0')


Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
